In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from sklearn import preprocessing
import math
from dgl.nn import SAGEConv

#### Set Path

In [4]:
current_dir = "/content/drive/MyDrive/GML Final Project/"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "Lipophilicity.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [5]:
""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, transform=None, train=False, scaler=None, scaler_regression=None):
      #The train attribute is set to the value of the train argument, indicating if the dataset is for training or not.
            self.train = train
            self.scaler = scaler
      #The data_set attribute is assigned the loaded graphs from the specified address using dgl.load_graphs.
            self.data_set, train_labels_masks_globals = dgl.load_graphs(address+".bin")
            num_graphs = len(self.data_set)
      #The labels, masks, and globals are extracted from train_labels_masks_globals obtained from loading the graphs.
      #They are reshaped into 2-dimensional tensors, where each row corresponds to the labels, masks, or globals of a graph.
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
      #The transform attribute is assigned the value of the transform argument, which can be used for data transformation if provided.
            self.transform = transform
      #The scaler_regression attribute is assigned the value of the scaler_regression argument, indicating whether a scaler should be used for regression labels or not.
            self.scaler_regression = scaler_regression
#The scaler_method method is defined to create and return a scaler object. If self.
#train is True, it fits a StandardScaler to the labels and assigns it to the self.scaler attribute. The scaler is returned.
    def scaler_method(self):
        if self.train:
            scaler = preprocessing.StandardScaler().fit(self.labels)
            self.scaler = scaler
        return self.scaler
#The __len__ method returns the number of graphs in the dataset.
    def __len__(self):
        return len(self.data_set)
#The __getitem__ method retrieves an item from the dataset at the given index (idx). There are two different paths based on whether self.scaler_regression is True or not:
    def __getitem__(self, idx):
        if self.scaler_regression:
            """ With Scaler"""
            return  self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.masks[idx], self.globals[idx]
        else:
            """ Without Scaler """
            return  self.data_set[idx], self.labels[idx].float(), self.masks[idx], self.globals[idx]


#### Defining Train, Validation, and Test Set

In [6]:
scaler = {}
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetReg(address=path_data_temp+"_train", train=True)
scaler = train_set.scaler_method()
val_set = DGLDatasetReg(address=path_data_temp+"_val", scaler=scaler)
test_set = DGLDatasetReg(address=path_data_temp+"_test", scaler=scaler)

print(len(train_set), len(val_set), len(test_set))


3360 420 420


#### Data Loader

In [7]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs

    ### This line extracts the graphs from each tuple in the batch and creates a list of individual graphs.
    graphs = [e[0] for e in batch]
    ### This line uses the dgl.batch() function to combine the individual graphs into a single graph batch g
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension

    ### This line extracts the labels from each tuple in the batch and creates a list of individual label tensors.
    labels = [e[1] for e in batch]
    ### This line uses torch.stack() to concatenate the individual label tensors along a new dimension (dimension 0) and create a single tensor labels representing the labels for the entire batch.
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension

    ### This line extracts the masks from each tuple in the batch and creates a list of individual mask tensors.
    masks = [e[2] for e in batch]
    ### This line uses torch.stack() to concatenate the individual mask tensors along a new dimension (dimension 0) and create a single tensor masks representing the masks for the entire batch.
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension

    ### This line extracts the global features from each tuple in the batch and creates a list of individual global feature tensors.
    globals = [e[3] for e in batch]
    ### This line uses torch.stack() to concatenate the individual global feature tensors along a new dimension (dimension 0) and create a single tensor globals representing the global features for the entire batch.
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [8]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

In [9]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


In [ ]:
dgl.use_libxsmm(False)

class CustomModule(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(CustomModule, self).__init__()

        self.linear1 = nn.Linear(in_feat * 2, 256)
        self.linear2 = nn.Linear(256, out_feat)
        self.relu = nn.ReLU()

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.update_all(
                message_func=fn.copy_u('h', 'm'),
                reduce_func=fn.max("m", "A"),
            )
            A = g.ndata["A"]
            concat = torch.cat([h, A], dim=1)
            output = self.linear1(concat)
            output = self.relu(output)
            output = self.linear2(output)
            output = self.relu(output)
            return output


In [ ]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.Custom1 = CustomModule(self.node_feature_size, self.hidden_size)
        self.Custom2 = CustomModule(self.hidden_size, self.hidden_size)
        self.Custom3 = CustomModule(self.hidden_size, self.hidden_size)
        self.Custom4 = CustomModule(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = mol_dgl_graph.ndata["v"]
        h = self.Custom1(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.Custom2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.Custom3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.Custom4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


In [10]:
def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
  model.eval()
  loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
  final_loss = 0
  state = torch.get_rng_state()
  with torch.no_grad():
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
       prediction = model(mol_dgl_graph, globals)
       prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
       labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
       loss = loss_sum(prediction, labels)
       final_loss += loss.item()

    final_loss /= val_size
    final_loss = math.sqrt(final_loss)

  return final_loss / num_tasks

In [11]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

In [12]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [13]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 100000
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader,scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [14]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [24]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 5.604 | Valid Score: 2.102
 
Epoch: 1/100 | Best Valid Score Until Now: 2.102 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.965 | Valid Score: 1.520
 
Epoch: 2/100 | Best Valid Score Until Now: 1.520 

Save checkpoint
Epoch: 3/100 | Training Loss: 1.683 | Valid Score: 1.464
 
Epoch: 3/100 | Best Valid Score Until Now: 1.464 

Save checkpoint
Epoch: 4/100 | Training Loss: 1.561 | Valid Score: 1.434
 
Epoch: 4/100 | Best Valid Score Until Now: 1.434 

Save checkpoint
Epoch: 5/100 | Training Loss: 1.462 | Valid Score: 1.399
 
Epoch: 5/100 | Best Valid Score Until Now: 1.399 

Save checkpoint
Epoch: 6/100 | Training Loss: 1.412 | Valid Score: 1.385
 
Epoch: 6/100 | Best Valid Score Until Now: 1.385 

Save checkpoint
Epoch: 7/100 | Training Loss: 1.378 | Valid Score: 1.383
 
Epoch: 7/100 | Best Valid Score Until Now: 1.383 

Save checkpoint
Epoch: 8/100 | Training Loss: 1.345 | Valid Score: 1.364
 
Epoch: 8/100 | Best Valid Score Until Now: 